# Мультикласова класифікація

У цьому домашньому завданні ми будемо працювати з наступним набором даних:
https://archive.ics.uci.edu/ml/datasets/car+evaluation

Ми передбачимо клас автомобілів відповідно до його характеристик. Клас може бути unacceptable, acceptable, good, very good.

Ми завантажуємо набір даних з hw_5_data.csv, наданого вам для виконання домашнього завдання.

0. Розташуйте всі імпорти під цією секцією.

In [ ]:
import warnings

warnings.filterwarnings('ignore')

1.1. Завантажте набір даних у змінну df за допомогою Pandas. Виведіть перші 5 записів та форму набору даних.

In [ ]:
# ваш код тут

1.2. Набір даних не має імен стовпців. Дайте наступні імена стовпцям:
`['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']`

In [ ]:
# ваш код тут

2. Виведіть кількість унікальних значень кожної з незалежних змінних програмно. Формат числовий, а не візуальний. Приклад відповіді для одного стовпця:
```
med      432
high     432
vhigh    432
low      432
Name: buying, dtype: int64
```
Необхідно, щоб були назви стовпців, назви значень та кількість рядків із цим значенням.

In [ ]:
# ваш код тут

3. Виведіть скільки рядків відповідають кожному із значень цільової змінної. Дайте відповіді на два наступні запитання:   
  3.1. Ця змінна ординальна (порядкова) чи номінальна? Чому?
  
  3.2. Класифікація у цьому випадку буде збалансована чи незбалансована? Чому?

In [ ]:
# ваш код тут

>ваша відповідь тут

4. Перевірте, чи відсутні значення у всіх змінних завантажених набору даних.

In [ ]:
# ваш код тут

5.1. Запишіть у змінну `x` всі незалежні змінні, у змінну `y` - цільову.


In [ ]:
# ваш код тут

5.2. Розбийте дані на тренувальний та тестовий набори (ми знову ігноруємо валідацію для спрощення завдання). Визначте random_state рівним 42 та test_size рівним 0.33.

In [ ]:
x_train_df, x_test_df, y_train, y_test = ...

5.3. Виведіть форму та перші 5 записів отриманих x_train_df, x_test_df.

In [ ]:
# ваш код тут

6.1. Виведіть типи даних усіх змінних у x_train.

In [ ]:
# ваш код тут

6.2. Усі незалежні змінні у нас ординальні. Закодуйте всі незалежні змінні за допомогою відповідного методу з sklearn.preprocessing.  
Вам необхідно встановити правильний порядок значень для кожної змінної при кодуванні так, щоб значення після кодування були впорядковані в порядку зростання.
Кодування значень {'high':1,'low':2,'med':3} буде ввжатись неправильним виконанням завдання.
Подумайте, як коректно застосовувати енкодер на трен. та тест. даних.
Результат кодування x_train_df та x_test_df запишіть відповідно у x_train, x_test.

In [ ]:
# ваш код тут

6.3. Виведіть перші 5 записів x_train, x_test.

In [ ]:
# ваш код тут

6.4. Виведіть отримані категорії енкодера.

In [ ]:
# ваш код тут

7.1. Навчіть на отриманих даних логістичну регресію c random_state=42 та іншими параметрами по замовчуванню.

In [ ]:
# ваш код тут

7.2. Зробіть прогнози за допомогою отриманої моделі за допомогою методу predict. 

In [ ]:
# ваш код тут

7.3. Виведіть метрики якості за допомогою classification_report.

In [ ]:
# ваш код тут

Accuracy у цьому пункті має бути близько 0.82. 

7.4. Яку метрику якості ви рекомендуєте використовувати для порівняння різних моделей та вибору найкращої? Чому? Метрика - це одне число, що характеризує якість передбачень моделі.

>ваша відповідь тут

8.1. Навчіть Decision Tree з sklearn з глибиною 4 за тими ж даними. Обчисліть прогнози, знайдіть метрику якості на test даних. Порівняйте з регресією логістики. Яка модель краща?

In [ ]:
# ваш код тут

8.2. Виведіть параметри моделі Decision Tree.

In [ ]:
# ваш код тут

<font color='blue'>**[Optional]**</font> 8.3. Візуалізуйте отримане дерево в будь-якому з існуючих методів. Зображення повинно мати графік з критеріями у вузлах, класах та кількості елементів у кожному аркуші.

In [ ]:
# ваш код тут

9.1. Виконайте пошук гіперпараметрів для DecisionTree за сіткою за допомогою sklearn. Встановіть сітку з параметрів самостійно на основі припущень, що може покращити роботу дерева.
Вам вдалося знайти модель, яка працює краще, ніж побудована у пункті 8.1?

In [ ]:
# ваш код тут

9.2. Виведіть claasification report фінальної моделі пошуку гіперпараметрів.

In [ ]:
# ваш код тут

9.3. Виведіть матрицю помилок для кращої моделі у пошуку гіперпараметрів. Відобразіть матрицю візуально (у вигляді графіка із зафарбованими за значеннями клітинами або у вигляді таблиці із зафарбовуванням фону).

In [ ]:
# ваш код тут

9.4. Зробіть висновок, які класи, з якими моделями найбільше плутаються?

>ваша відповідь тут